# Homework 3 - Application Building

- Python Computing for Data Science (2022)

- Due Tuesday Feb 15 (8pm)

### Casey Lam, casey_lam@berkeley.edu

### Collaborators: TBD

## CalCalc

Write a module called `CalCalc`, with a method called `calculate` that evaluates any string passed to it, and can be used from either the command line (using `argparse` with reasonable flags) or imported within Python. Feel free to use something like `eval()`, but be aware of some of the nasty things it can do, and make sure it doesn’t have too much power:  http://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html. Perhaps explore the use of `numexpr` to constrain the landscape of possible uses to math expressions.

EXAMPLE:
```bash
$ python CalCalc.py -s '34*28'
$ 952
```
 AND, from within Python
 
```python
>>> from CalCalc import calculate
>>> calculate('34*20')
>>> 952
```

In [97]:
%%writefile calcalc/CalCalc.py

import argparse
import numexpr
import urllib
import requests
import pdb
import re

def calculate(in_str, return_float=False):
    # Evaluate remotely using wolfram.
    if return_float:
        url_str = urllib.parse.quote_plus(in_str)
        app_id = 'Q9RQK4-QK54QKTJ72'
        url_wolfram = 'https://api.wolframalpha.com/v2/query?input=' + url_str + '&appid=' + app_id + '&output=json&scanner=Data'
        # url_wolfram = 'https://api.wolframalpha.com/v2/query?input=' + url_str + '&appid=' + app_id
        #url_wolfram = 'https://api.wolframalpha.com/v1/result?i=How+many+ounces+are+in+a+gallon%3F&appid=DEMO'
        
        try:
            # Need some condition for "about"
            # No short answer available
            # hundred, ten, thousand, etc...
            answer = requests.get(url_wolfram)
            answer_text = answer.json()['queryresult']['pods'][0]['subpods'][0]['img']['title']
            print('Answer (direct from Wolfram): ', answer_text)
            #answer_text.rstrip().split('(') 
            #answer_text.rstrip().split('\n') 
            answer_text = answer_text.split('\n', 1)[0] # skip parenthetical clarifications, units.
            answer_text = answer_text.split(' (', 1)[0] # skip blah blah details.
            answer_text = answer_text.split('to', 1)[0] # give lower range
            #print(answer_text)
            #answer_text = requests.get(url_wolfram).text
            #print(answer_text)
            answer_text = answer_text.replace('×', '*')
            answer_text = answer_text.replace('^', '**')
            #answer_text = answer_text.replace(' to the ', '**')
            #answer_text = answer_text.replace(' times ', '*')
            answer_text = re.sub('[^1234567890*.]', '', answer_text)
            print('Answer (cleaned up): ', answer_text)
            answer = float(numexpr.evaluate(answer_text))
            print('Answer (float): ', answer)
            return answer
        
        except:
            print('This question\'s answer isn\'t convertable to a string. \n' + 
                  'Try rephrasing your question (e.g. specify units of the result).')

    # Evaluate locally using python.
    else:
        try:
            print('input: ', in_str)
            print('numexpr: ', numexpr.evaluate(in_str))
            answer = float(numexpr.evaluate(in_str))
            print('floated: ', answer)
            return answer
        except:
            print('This expression can\'t be evaluated numerically. \n' + 
                  'Did you mean to evaluate with wolfram? \n' +
                  'If not, check your question for typos.')
        
def test1():
    assert abs(4. - calculate('2**2')) < 0.001
    
def test2():
    assert abs(4. - calculate('2**2')) < 0.001
    
def test3():
    assert abs(4. - calculate('2**2')) < 0.001
    
def test4():
    assert abs(4. - calculate('2**2')) < 0.001
    
def test5():
    assert abs(4. - calculate('2**2')) < 0.001
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Write something useful here.')
    parser.add_argument('-s', action='store', dest='question_python', 
                        help='Numbers', default=None)
    parser.add_argument('-w', action='store', dest='question_wolfram',
                        help='Words', default=None)
    # Maybe edit this, so that instead it will return as string.

    results = parser.parse_args()
    
#    print(results.question_python)
#    print(results.question_wolfram)
    
    if (results.question_python != None) & (results.question_wolfram != None):
        raise Exception('Make up your mind! You can only set one flag.')

    if results.question_wolfram is None:
        return_float=False
        question = results.question_python
    else:
        return_float=True     
        question = results.question_wolfram
        
    calculate(question, return_float=return_float)

Overwriting calcalc/CalCalc.py


In [99]:
# !python calcalc/CalCalc.py -w 'mass of the moon in kg'
# !python calcalc/CalCalc.py -w 'how many pandas'
# !python calcalc/CalCalc.py -w 'how many countries'
# !python calcalc/CalCalc.py -w 'price of gold'
# !python calcalc/CalCalc.py -w 'price of gold in dollars per oz'
# !python calcalc/CalCalc.py -w 'CHICKENS!!!!!'
!python calcalc/CalCalc.py -w 'how many people on earth' # This does not behave right. Need dictionary of values.
!python calcalc/CalCalc.py -w 'how many grains of sand' # This requires the 1 to be 1. to work.
!python calcalc/CalCalc.py -s 'mass of the moon in kg'
!python calcalc/CalCalc.py -s '1*10**100'
!python calcalc/CalCalc.py -s '1.*10**100'

Answer (direct from Wolfram):  7.3459×10^22 kg (kilograms)
Answer (cleaned up):  7.3459*10**22
Answer (float):  7.3459e+22
Answer (direct from Wolfram):  7.71 billion people (2019 estimate)
Answer (cleaned up):  7.71
Answer (float):  7.71
Answer (direct from Wolfram):  (1×10^20 to 1×10^24)
Answer (cleaned up):  1*10**20
This question's answer isn't convertable to a string. 
Try rephrasing your question (e.g. specify units of the result).
input:  mass of the moon in kg
This expression can't be evaluated numerically. 
Did you mean to evaluate with wolfram? 
If not, check your question for typos.
input:  1*10**100
This expression can't be evaluated numerically. 
Did you mean to evaluate with wolfram? 
If not, check your question for typos.
input:  1.*10**100
numexpr:  1e+100
floated:  1e+100


In [63]:
import urllib
import requests 

url_str = urllib.parse.quote_plus('number of countries')
app_id = 'Q9RQK4-QK54QKTJ72'
url_wolfram = 'https://api.wolframalpha.com/v2/query?input=' + url_str + '&appid=' + app_id + '&output=json&scanner=Data'
#url_wolfram = 'https://api.wolframalpha.com/v1/result?i=How+many+ounces+are+in+a+gallon%3F&appid=DEMO'

answer = requests.get(url_wolfram)

In [72]:
from importlib import reload
reload(CalCalc)
CalCalc.calculate('how many countries', return_float=True)

> /home/jovyan/python-ay250-homeworks/hw_3/calcalc/CalCalc.py(25)calculate()
     23             answer_text = answer.json()['queryresult']['pods'][0]['subpods'][0]['img']['title']
     24             pdb.set_trace()
---> 25             print('Answer (direct from Wolfram): ', answer_text)
     26             #answer_text.rstrip().split('(') # skip parenthetical clarifications, units.
     27             #answer_text.rstrip().split('\n') # skip blah blah details.

ipdb> l
     20             # No short answer available
     21             # hundred, ten, thousand, etc...
     22             answer = requests.get(url_wolfram)
     23             answer_text = answer.json()['queryresult']['pods'][0]['subpods'][0]['img']['title']
     24             pdb.set_trace()
---> 25             print('Answer (direct from Wolfram): ', answer_text)
     26             #answer_text.rstrip().split('(') # skip parenthetical clarifications, units.
     27             #answer_text.rstrip().split('\n') # sk

ipdb> answer_text
'206'
ipdb> q
This question's answer isn't convertable to a string. \Try rephrasing your question (e.g. specify units of the result).


In [28]:
url_wolfram

'https://api.wolframalpha.com/v2/query?input=number+of+countries&appid=Q9RQK4-QK54QKTJ72&output=json&scanner=Data'

In [29]:
for key in answer.json()['queryresult'].keys():
    print(key)
    print(answer.json()['queryresult'][key])
    print('')

success
True

error
False

numpods
1

datatypes


timedout


timedoutpods


timing
0.628

parsetiming
0.47800000000000004

parsetimedout
False

recalculate


id
MSP5109143043e73cih7e21000039hcag778a435cbf

host
https://www6b3.wolframalpha.com

server
11

related
https://www6b3.wolframalpha.com/api/v1/relatedQueries.jsp?id=MSPa5110143043e73cih7e21000037a3fc6h96a81b115481992349747128482

version
2.6

inputstring
number of countries

pods
[{'title': 'Result', 'scanner': 'Data', 'id': 'Result', 'position': 200, 'error': False, 'numsubpods': 1, 'primary': True, 'subpods': [{'title': '', 'img': {'src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP5111143043e73cih7e210000529d7a1bg100c7ff?MSPStoreType=image/gif&s=11', 'alt': '206\n(according to Article 1 of the Montevideo Convention of 1933, in which a state must have: (1) a permanent population, (2) a defined territory, (3) a government, and (4) a capacity to enter into relations with the other states)', 'title': '206\n(according to Arti

In [65]:
import sys
sys.path.append('./calcalc/')
import CalCalc

### Add Wolfram

To make this more awesome, have your function interact with the Wolfram|Alpha API to ask it what it thinks of the difficult questions.  To make this work, experiment with `urllib2` and a URL like this:
'http://api.wolframalpha.com/v2/query?input=XXXXX&appid=UAGAWR-3X6Y8W777Q'
where you replace the XXXXX with what you want to know.  NOTE: the ‘&appid=UAGAWR-3X6Y8W777Q’ part is vital; it is a W|A AppID I got for the class.  Feel free to use that one, or you can get your own and read more about the API, here:   http://products.wolframalpha.com/api/
And you can explore how it works here:  http://products.wolframalpha.com/api/explorer.html

EXAMPLE:

```bash
$ python CalCalc.py -w 'mass of the moon in kg'
7.3459e+22
```

AND, from within Python

```python
>>> from CalCalc import calculate
>>> calculate('mass of the moon in kg',  return_float=True) * 10
>>> 7.3459e+23
```

## Adding it to Github

Start a github project for CalCalc. Include a setup.py, README.txt, LICENSE.txt, MANIFEST.in, etc. and turn your module into a proper Python Distribution, so that we can install it and use it. See https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/ 

Example Folder Hierarchy:
```bash
Your_Homework3_Folder/calcalc
                      |--> CalCalc.py
                      |--> __init__.py
Your_Homework3_Folder/setup.py
Your_Homework3_Folder/README.txt
...
```
Include at least 5 test functions in CalCalc.py, and test with `pytest`, to make sure it behaves the way you think it should.

EXAMPLE `CalCalc.py`:
```python
# ...
def calculate([...]):
    [...]

def test_1():
    assert abs(4. - calculate('2**2')) < 0.001
```

When grading, we will create a virtual environment and attempt to install your module by running:

```bash
pip install build
```

### CalCalc on CI

Get your project working with GitHub Actions and make sure your tests are run and pass. Give us a link to you GH actions for your site here (e.g. https://github.com/profjsb/PyAdder/actions):

### **(Bonus/Extra Credit)** 

  Get your project working on Azure, AWS or Google Compute Cloud with a Flask front-end. You can use the example from class as a template. Start a VM on one of these PaaS. A user should be able to submit their calcalc query on a form (hosted on your VM) and get the result back.

You should be able to add an `app.py` (with Flask) into your CalCalc project. Be sure to open up the port on the VM that you are serving on. Let us know the URL to your app here: